#### 1. Preparation

In [1]:
#
import os
import pathlib

import numpy as np
import pandas as pd

import xarray as xr
import rioxarray as rxr
import rasterio as rio

import geopandas as gpd
from shapely.geometry import LineString, Point, MultiPoint, box

import matplotlib.pyplot as plt
import seaborn as sns

import netCDF4

In [2]:
# Get new path
# Create a path to the folder
# that contains file "VCSN_waikanae_200501_disaggdaily_500m_nztm_masked_sec.nc"
main_dir = r"S:\Grid_Ori_bigdata_005"

#### 2. Process data

In [3]:
# Get data
# Get new x and new y
rainfall_cyp_xr = xr.open_dataset(fr"{main_dir}\VCSN_waikanae_200501_disaggdaily_500m_nztm_masked_sec.nc")

# Change variable name
rainfall_cyp_xr['rainfall_depth'] = rainfall_cyp_xr['depth']
rainfall_cyp_xr = rainfall_cyp_xr.drop('depth')

# Assign CRS
rainfall_cyp_xr.rio.write_crs('epsg:2193', inplace=True)

<xarray.Dataset>
Dimensions:         (time: 120, x: 37, y: 42)
Coordinates:
  * time            (time) timedelta64[ns] 00:00:00 01:00:00 ... 4 days 23:00:00
  * x               (x) float64 1.765e+06 1.766e+06 ... 1.783e+06 1.783e+06
  * y               (y) float64 5.459e+06 5.46e+06 ... 5.479e+06 5.48e+06
    spatial_ref     int32 0
Data variables:
    rainfall_depth  (time, y, x) float32 ...
Attributes:
    Conventions:               COARDS, CF-1.5
    title:                     Produced by grdmath
    history:                   Sun Jan 30 20:31:47 2022: ncatted -a units,tim...
    description:               
    GMT_version:               5.4.5 [64-bit]
    node_offset:               1
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...
    nco_openmp_thread_number:  1

In [4]:
# Clip data
geometry_domain_clip = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [1767790 - 1150, 5471400 - 1150],
            [1767790 - 1150, 5475880 + 1150],
            [1774510 + 1150, 5475880 + 1150],
            [1774510 + 1150, 5471400 + 1150],
            [1767790 - 1150, 5471400 - 1150]
        ]]
    }
]

In [5]:
rainfall_clip_first = rainfall_cyp_xr.rio.clip(geometry_domain_clip)

#### 3. Interpolation

In [6]:
# Check boundaries
ori_rain_bound = rainfall_clip_first.rio.bounds()
ori_rain_bound

(1766709.4594594594, 5470250.0, 1775466.2162162163, 5477250.0)

In [7]:
%%time
# Get new x and new y
new_x = np.arange(ori_rain_bound[0]+5, ori_rain_bound[2], step=10)
new_y = np.arange(ori_rain_bound[1]+5, ori_rain_bound[3], step=10)

# Get interpolation
rainfall_interpolation = rainfall_cyp_xr.interp(x=new_x, y=new_y, method='nearest')
rainfall_interpolation.rio.write_crs('epsg:2193', inplace=True)
rainfall_interpolation.to_netcdf(fr"{main_dir}\rainfall_interpolation_nearest.nc", format="NETCDF4", engine="netcdf4")

CPU times: total: 641 ms
Wall time: 3.47 s
